# ARIMA Forecasting to predict $ET_0$

Libs required

In [1]:
%matplotlib inline

import pandas as pd
import numpy as np
import warnings
from sklearn import metrics as mt
from pyramid.arima import auto_arima

from matplotlib import pyplot as plt
import plotly
import plotly.plotly as py
import plotly.graph_objs as go

warnings.filterwarnings("ignore")

plotly.tools.set_credentials_file(username='Dieinison', api_key='3rLjaomkz95dUHsgfAi6')

In [2]:
et0_treino_ = pd.read_pickle('./dataset/completed_split/label_train_.pkl') 
et0_test = pd.read_pickle('./dataset/completed_split/label_test.pkl')
et0_completed = pd.read_pickle('./dataset/label.pkl')

## Hyper-parameters optimization

In [4]:
sarima_model = auto_arima(et0_completed, start_p=1, start_q=1, max_p=5, max_q=5, m=12,
                          start_P=0, seasonal=True, d=1, D=1, trace=True,
                          error_action='ignore',  # don't want to know if an order does not work
                          suppress_warnings=True,  # don't want convergence warnings
                          stepwise=True)  # set to stepwise

Fit ARIMA: order=(1, 1, 1) seasonal_order=(0, 1, 1, 12); AIC=4339.978, BIC=4365.172, Fit time=5.620 seconds
Fit ARIMA: order=(0, 1, 0) seasonal_order=(0, 1, 0, 12); AIC=5342.046, BIC=5352.124, Fit time=0.234 seconds
Fit ARIMA: order=(1, 1, 0) seasonal_order=(1, 1, 0, 12); AIC=4826.634, BIC=4846.789, Fit time=1.266 seconds
Fit ARIMA: order=(0, 1, 1) seasonal_order=(0, 1, 1, 12); AIC=4401.206, BIC=4421.361, Fit time=2.118 seconds
Fit ARIMA: order=(1, 1, 1) seasonal_order=(1, 1, 1, 12); AIC=4345.737, BIC=4375.970, Fit time=6.594 seconds
Fit ARIMA: order=(1, 1, 1) seasonal_order=(0, 1, 0, 12); AIC=5022.425, BIC=5042.580, Fit time=1.958 seconds
Fit ARIMA: order=(1, 1, 1) seasonal_order=(0, 1, 2, 12); AIC=4341.965, BIC=4372.198, Fit time=14.012 seconds
Fit ARIMA: order=(1, 1, 1) seasonal_order=(1, 1, 2, 12); AIC=4343.498, BIC=4378.770, Fit time=8.170 seconds
Fit ARIMA: order=(2, 1, 1) seasonal_order=(0, 1, 1, 12); AIC=nan, BIC=nan, Fit time=nan seconds
Fit ARIMA: order=(1, 1, 0) seasonal_ord

In [5]:
sarima_model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                                 Statespace Model Results                                 
==========================================================================================
Dep. Variable:                                  y   No. Observations:                 1153
Model:             SARIMAX(2, 1, 2)x(0, 1, 1, 12)   Log Likelihood               -2161.388
Date:                            Wed, 27 Feb 2019   AIC                           4336.775
Time:                                    22:26:37   BIC                           4372.047
Sample:                                         0   HQIC                          4350.095
                                           - 1153                                         
Covariance Type:                              opg                                         
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
intercept     -0.0003      0.000     -0.657      0.511      -0.001       0.001
ar.L1         -0.3613      0.161     -2.247      0.025      -0.676      -0.046
ar.L2          0.3467      0.056      6.169      0.000       0.237       0.457
ma.L1         -0.1362      0.164     -0.831      0.406      -0.458       0.185
ma.L2         -0.6491      0.136     -4.756      0.000      -0.917      -0.382
ma.S.L12      -0.9769      0.014    -71.271      0.000      -1.004      -0.950
sigma2         2.5159      0.096     26.320      0.000       2.329       2.703
===================================================================================
Ljung-Box (Q):                       60.44   Jarque-Bera (JB):                18.29
Prob(Q):                              0.02   Prob(JB):                         0.00
Heteroskedasticity (H):               0.97   Skew:                            -0.18
Prob(H) (two-sided):                  0.74   Kurtosis:                         3.50
===================================================================================

Warnings:
[1] Covariance matrix calculated using the outer product of gradients (complex-step).
"""

## Time Series Forecasting

In [6]:
sarima_model.fit(et0_treino_)

ARIMA(callback=None, disp=0, maxiter=50, method=None, order=(2, 1, 2),
   out_of_sample_size=0, scoring='mse', scoring_args={},
   seasonal_order=(0, 1, 1, 12), solver='lbfgs', start_params=None,
   suppress_warnings=True, transparams=True, trend='c')

In [7]:
et0_pred = sarima_model.predict(n_periods=len(et0_test))

print('RMSE = {}\nMAE  = {}\n'.format(
    np.sqrt(mt.mean_squared_error(et0_test, et0_pred)), 
    mt.mean_absolute_error(et0_test, et0_pred)))

RMSE = 3.5929050065633175
MAE  = 3.022643831164893



In [8]:
et0_completed.describe()

,ETo
count,1153.000000
mean,6.871174
std,2.565511
min,0.852370
25%,5.173478
50%,6.859215
75%,8.620300
max,13.723929


In [9]:
trace0 = go.Scatter(
    x = et0_test.index,
    y = et0_test.ETo.values,
    mode = 'lines',
    name = 'ET0'
)

trace1 = go.Scatter(
    x = et0_test.index,
    y = et0_pred,
    mode = 'lines',
    name = 'Predicted'
)

data = [trace0, trace1]

fig = go.Figure(data=data)
py.iplot(fig, filename='SARIMA Forecasting - Time Series completed')

## Estação Chuvosa

In [10]:
et0_rain_treino_ = pd.read_pickle('./dataset/rain_split/label_chuva_train_.pkl') 
et0_rain_test = pd.read_pickle('./dataset/rain_split/label_chuva_test.pkl')
et0_rain_completed = pd.read_pickle('./dataset/label_estacao_chuvosa.pkl')

In [11]:
Sarima_model = auto_arima(et0_rain_completed, start_p=1, start_q=1, max_p=5, max_q=5, m=12,
                          start_P=0, seasonal=True, d=1, D=1, trace=True,
                          error_action='ignore',  # don't want to know if an order does not work
                          suppress_warnings=True,  # don't want convergence warnings
                          stepwise=True)  # set to stepwise

Fit ARIMA: order=(1, 1, 1) seasonal_order=(0, 1, 1, 12); AIC=1480.174, BIC=1499.795, Fit time=1.887 seconds
Fit ARIMA: order=(0, 1, 0) seasonal_order=(0, 1, 0, 12); AIC=1755.252, BIC=1763.100, Fit time=0.060 seconds
Fit ARIMA: order=(1, 1, 0) seasonal_order=(1, 1, 0, 12); AIC=1639.855, BIC=1655.552, Fit time=0.388 seconds
Fit ARIMA: order=(0, 1, 1) seasonal_order=(0, 1, 1, 12); AIC=1501.875, BIC=1517.572, Fit time=1.074 seconds
Fit ARIMA: order=(1, 1, 1) seasonal_order=(1, 1, 1, 12); AIC=1481.365, BIC=1504.910, Fit time=2.069 seconds
Fit ARIMA: order=(1, 1, 1) seasonal_order=(0, 1, 0, 12); AIC=1672.859, BIC=1688.556, Fit time=0.871 seconds
Fit ARIMA: order=(1, 1, 1) seasonal_order=(0, 1, 2, 12); AIC=1481.251, BIC=1504.797, Fit time=6.078 seconds
Fit ARIMA: order=(1, 1, 1) seasonal_order=(1, 1, 2, 12); AIC=1484.488, BIC=1511.958, Fit time=8.831 seconds
Fit ARIMA: order=(2, 1, 1) seasonal_order=(0, 1, 1, 12); AIC=nan, BIC=nan, Fit time=nan seconds
Fit ARIMA: order=(1, 1, 0) seasonal_orde

In [12]:
Sarima_model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                                 Statespace Model Results                                 
==========================================================================================
Dep. Variable:                                  y   No. Observations:                  387
Model:             SARIMAX(3, 1, 3)x(0, 1, 1, 12)   Log Likelihood                -728.588
Date:                            Wed, 27 Feb 2019   AIC                           1475.177
Time:                                    22:29:58   BIC                           1510.495
Sample:                                         0   HQIC                          1489.200
                                            - 387                                         
Covariance Type:                              opg                                         
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
intercept     -0.0030      0.004     -0.854      0.393      -0.010       0.004
ar.L1         -1.3457      0.071    -18.898      0.000      -1.485      -1.206
ar.L2          0.1461      0.120      1.215      0.224      -0.090       0.382
ar.L3          0.5419      0.066      8.215      0.000       0.413       0.671
ma.L1          0.9796      0.056     17.498      0.000       0.870       1.089
ma.L2         -0.8099      0.067    -12.056      0.000      -0.942      -0.678
ma.L3         -0.8701      0.053    -16.528      0.000      -0.973      -0.767
ma.S.L12      -0.9998     10.272     -0.097      0.922     -21.133      19.133
sigma2         2.5962     26.580      0.098      0.922     -49.500      54.692
===================================================================================
Ljung-Box (Q):                       32.44   Jarque-Bera (JB):                 1.07
Prob(Q):                              0.80   Prob(JB):                         0.58
Heteroskedasticity (H):               0.72   Skew:                            -0.13
Prob(H) (two-sided):                  0.07   Kurtosis:                         3.04
===================================================================================

Warnings:
[1] Covariance matrix calculated using the outer product of gradients (complex-step).
"""

In [13]:
Sarima_model.fit(et0_rain_treino_)

ARIMA(callback=None, disp=0, maxiter=50, method=None, order=(3, 1, 3),
   out_of_sample_size=0, scoring='mse', scoring_args={},
   seasonal_order=(0, 1, 1, 12), solver='lbfgs', start_params=None,
   suppress_warnings=True, transparams=True, trend='c')

In [14]:
et0_rain_pred = Sarima_model.predict(n_periods=len(et0_rain_test))

print('RMSE = {}\nMAE  = {}\n'.format(
    np.sqrt(mt.mean_squared_error(et0_rain_test, et0_rain_pred)), 
    mt.mean_absolute_error(et0_rain_test, et0_rain_pred)))

RMSE = 2.5355862160044
MAE  = 2.120414751957596



In [15]:
et0_rain_completed.describe()

,ETo
count,387.000000
mean,5.272518
std,2.079660
min,0.852370
25%,3.691112
50%,5.447556
75%,6.813897
max,10.320710


In [16]:
trace0 = go.Scatter(
    x = et0_rain_test.index,
    y = et0_rain_test.ETo.values,
    mode = 'lines',
    name = 'ET0'
)

trace1 = go.Scatter(
    x = et0_rain_test.index,
    y = et0_rain_pred,
    mode = 'lines',
    name = 'Predicted'
)

data = [trace0, trace1]

fig = go.Figure(data=data)
py.iplot(fig, filename='SARIMA Forecasting - Time Series Raining Station')

## Estação de seca

In [17]:
et0_seca_treino_ = pd.read_pickle('./dataset/drought_split/label_seca_train_.pkl') 
et0_seca_test = pd.read_pickle('./dataset/drought_split/label_seca_test.pkl')
et0_seca_completed = pd.read_pickle('./dataset/label_estacao_seca.pkl')

In [18]:
sarima_model = auto_arima(et0_seca_completed, start_p=1, start_q=1, max_p=5, max_q=5, m=12,
                          start_P=0, seasonal=True, d=1, D=1, trace=True,
                          error_action='ignore',  # don't want to know if an order does not work
                          suppress_warnings=True,  # don't want convergence warnings
                          stepwise=True)  # set to stepwise

Fit ARIMA: order=(1, 1, 1) seasonal_order=(0, 1, 1, 12); AIC=2870.444, BIC=2893.565, Fit time=3.913 seconds
Fit ARIMA: order=(0, 1, 0) seasonal_order=(0, 1, 0, 12); AIC=3518.004, BIC=3527.252, Fit time=0.524 seconds
Fit ARIMA: order=(1, 1, 0) seasonal_order=(1, 1, 0, 12); AIC=3151.537, BIC=3170.034, Fit time=4.498 seconds
Fit ARIMA: order=(0, 1, 1) seasonal_order=(0, 1, 1, 12); AIC=2903.604, BIC=2922.101, Fit time=2.435 seconds
Fit ARIMA: order=(1, 1, 1) seasonal_order=(1, 1, 1, 12); AIC=2872.286, BIC=2900.031, Fit time=5.277 seconds
Fit ARIMA: order=(1, 1, 1) seasonal_order=(0, 1, 0, 12); AIC=3300.487, BIC=3318.984, Fit time=1.365 seconds
Fit ARIMA: order=(1, 1, 1) seasonal_order=(0, 1, 2, 12); AIC=2875.046, BIC=2902.790, Fit time=12.036 seconds
Fit ARIMA: order=(1, 1, 1) seasonal_order=(1, 1, 2, 12); AIC=2873.740, BIC=2906.109, Fit time=8.814 seconds
Fit ARIMA: order=(2, 1, 1) seasonal_order=(0, 1, 1, 12); AIC=2867.815, BIC=2895.559, Fit time=4.477 seconds
Fit ARIMA: order=(2, 1, 0) 

In [19]:
sarima_model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                                 Statespace Model Results                                 
==========================================================================================
Dep. Variable:                                  y   No. Observations:                  766
Model:             SARIMAX(2, 1, 1)x(0, 1, 1, 12)   Log Likelihood               -1427.907
Date:                            Wed, 27 Feb 2019   AIC                           2867.815
Time:                                    22:33:21   BIC                           2895.559
Sample:                                         0   HQIC                          2878.503
                                            - 766                                         
Covariance Type:                              opg                                         
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
intercept     -0.0003      0.000     -1.161      0.246      -0.001       0.000
ar.L1          0.3876      0.040      9.617      0.000       0.309       0.467
ar.L2          0.1090      0.037      2.917      0.004       0.036       0.182
ma.L1         -0.9270      0.022    -42.594      0.000      -0.970      -0.884
ma.S.L12      -0.9756      0.025    -39.602      0.000      -1.024      -0.927
sigma2         2.4709      0.110     22.495      0.000       2.256       2.686
===================================================================================
Ljung-Box (Q):                       45.48   Jarque-Bera (JB):                42.83
Prob(Q):                              0.25   Prob(JB):                         0.00
Heteroskedasticity (H):               1.14   Skew:                            -0.27
Prob(H) (two-sided):                  0.31   Kurtosis:                         4.04
===================================================================================

Warnings:
[1] Covariance matrix calculated using the outer product of gradients (complex-step).
"""

In [20]:
sarima_model.fit(et0_seca_treino_)

ARIMA(callback=None, disp=0, maxiter=50, method=None, order=(2, 1, 1),
   out_of_sample_size=0, scoring='mse', scoring_args={},
   seasonal_order=(0, 1, 1, 12), solver='lbfgs', start_params=None,
   suppress_warnings=True, transparams=True, trend='c')

In [21]:
et0_seca_pred = sarima_model.predict(n_periods=len(et0_seca_test))

print('RMSE = {}\nMAE  = {}\n'.format(
    np.sqrt(mt.mean_squared_error(et0_seca_test, et0_seca_pred)), 
    mt.mean_absolute_error(et0_seca_test, et0_seca_pred)))

RMSE = 2.2494372936388047
MAE  = 1.8058874697672929



In [22]:
et0_seca_completed.describe()

,ETo
count,766.000000
mean,7.678850
std,2.404814
min,0.908164
25%,6.012210
50%,7.736559
75%,9.402421
max,13.723929


In [23]:
trace0 = go.Scatter(
    x = et0_seca_test.index,
    y = et0_seca_test.ETo.values,
    mode = 'lines',
    name = 'ET0'
)

trace1 = go.Scatter(
    x = et0_seca_test.index,
    y = et0_seca_pred,
    mode = 'lines',
    name = 'Predicted'
)

data = [trace0, trace1]

fig = go.Figure(data=data)
py.iplot(fig, filename='SARIMA Forecasting - Time Series Drought Station')